In [2]:
import numpy as np

In [17]:
def calculate_vertices(center, length, width, heading):
        """ Calculates the vertices given center, length, width and heading."""
        R_w_v = np.array([[np.cos(heading), -np.sin(heading)],
                          [np.sin(heading),  np.cos(heading)]])
        
        A = R_w_v@np.array([ length/2, width/2]) + center
        B = R_w_v@np.array([ length/2,-width/2]) + center
        C = R_w_v@np.array([-length/2,-width/2]) + center
        D = R_w_v@np.array([-length/2, width/2]) + center 
        return (A, B, C, D)

In [21]:
center = np.array([5, 5])
length = 4
width = 2
heading = np.pi/2


print(calculate_vertices(center, length, width, heading))

center = np.array([5, 5])
length = 4
width = 2
heading = np.pi/6


print(calculate_vertices(center, length, width, heading))

(array([4., 7.]), array([6., 7.]), array([6., 3.]), array([4., 3.]))
(array([6.23205081, 6.8660254 ]), array([7.23205081, 5.1339746 ]), array([3.76794919, 3.1339746 ]), array([2.76794919, 4.8660254 ]))


In [ ]:
from Object import *
import numpy as np
import time

class Vehicle(Object):
    def __init__(self, center: np.array, width: int = 1, lenght: int = 1, # Superparameters
                d=0.5, dt = 0.1, gamma=5e-7, alpha_max=1.2, v_max=8):     # Specific parameters
        super().__init__(self, center, width, lenght)

        """ Coordinate frame"""
        self.originVCF = Vector2(0.0, 0.0)
        self.verticesVCF = List[0]

        """ Current state """
        #self.speed = 0.0
        #self.norm_velocity = Vector2(1.0, 0.0)  # Direction of the vehicle
        self.psi = 0 # Heading
        self.X = np.array([[0], [0], [0], [0]]) # State vector
        self.psi = 0    # Heading
        self.alpha = 0  # Wheel angle. Throttle level can be derived from X[2:,:]
        self.omega = 0  # Current turning rate
        self.steering = 0.0
        self.throttle = 0.0

        """ vehicle structure """
        self.d = d

        """Dynamics parameters"""
        self.dt = dt
        self.tau_throttle = 1.0 # Parameter used in the throttle model
        self.tau_steering = 1.0 # Parameter used in the steering model
        self.K_a = self.dt / self.tau_steering # Gain parameter
        self.K_v = self.dt / self.tau_throttle # Gain parameter
        self.v_max = v_max      # Parameter used in the throttle model
        self.alpha_max = alpha_max
        self.k_max = 3.0        # Velocity signal parameter
        self.c_max = 0.5        # Steering signal parameter

        """Others"""
        self.gamma = gamma
        self.c = self.alpha_to_inverse_curve()

    @staticmethod
    def calculate_vertices(center, length, width, heading):
        """ Calculates the vertices given center, length, width and heading."""
        R_w_v = np.array([[np.cos(heading), -np.sin(heading)],
                          [np.sin(heading),  np.cos(heading)]])
        
        A = R_w_v@np.array([[], []]) + center

        return np.array([[center[0]-width/2, center[1]+length/2], # A
                         [center[0]+width/2, center[1]+length/2], # B
                         [center[0]+width/2, center[1]-length/2], # C
                         [center[0]-width/2, center[1]-length/2]])# D

    def throttle_model(self, r) -> float:
        # Linear map from throttle input to v_ref
        return self.k_max * r

    def steering_model(self, z) -> float:
        # Linear map from steering input to alpha_ref
        return self.c_max * z
    
    def alpha_to_inverse_curve(self):
        # Mapping from steering angle alpha, to inverse curve radius c = 1/R.
        return np.tan(self.alpha)/self.d

    def coordinated_turn_motion(self, v_ref: float, alpha_ref: float):
        """
        Running one discrete timestep of the motion model.
        """
        # Step 1: update u, alpha, find B_k
        self.alpha = self.alpha + self.K_a * (alpha_ref - self.alpha)
        self.c = self.alpha_to_inverse_curve()

        v_k = self.X[2:,:] # Current speed from state matrix
        u_k = self.K_v * (np.linalg.norm(v_ref)-np.linalg.norm(v_k))
        B_k = np.array([[0],
                        [0], 
                        [np.cos(self.psi)], 
                        [np.sin(self.psi)]])
    
        # Step 2: Choose A_k
        if np.abs(self.omega) > self.gamma:
            A_k = np.array([[1, 0, np.sin(self.omega*self.dt)/self.omega, -(1-np.cos(self.omega*self.dt))/self.omega],
                          [0, 1, (1-np.cos(self.omega*self.dt))/self.omega, np.sin(self.omega*self.dt)/self.omega],
                          [0, 0, np.cos(self.omega*self.dt), -np.sin(self.omega*self.dt)], 
                          [0, 0, np.sin(self.omega*self.dt), np.cos(self.omega*self.dt)]], dtype=np.ndarray)
        else: # In case omega ~ 0
            A_k = np.array([[1, 0, self.dt, 0],
                          [0, 1, 0, self.dt],
                          [0, 0, 1, 0], 
                          [0, 0, 0, 1]])
        
        # Step 3: Update state matrix
        self.X = A_k@self.X + B_k*u_k

        # Step 4: Update omega nad psi (based on theta)
        theta_next = np.linalg.norm(v_k)*self.dt*np.tan(self.alpha) / self.d # Turn angle
        self.psi = self.psi + theta_next # Heading in WC
        self.omega = np.linalg.norm(v_k)*np.tan(self.alpha) / self.d



    def brownian_motion_model(self, steps : int = 1000, v_ref = 3, alpha_ref = 0.0, var_vel = 0.5, var_alpha = 0.2, r_factor=0.02):
        """
        Random walk algorithm.
        """
        # Book keeping, for printing
        states = np.zeros((4, 1, steps))
        alphas = np.zeros((steps,))
        cs = np.zeros((steps,))
        v_refs = np.ones((steps,))*v_ref
        alpha_refs = np.ones((steps,))*alpha_ref
        psis = np.zeros((steps,))

        for i in range(steps):
            # Random driver:
            if (i*self.dt).is_integer: # Checks only on whole seconds
                if np.random.choice(a=[0,1], p=[1-r_factor, r_factor]): # Random motion has occured
                    choices = np.random.randint(0, 2, 2)
                    if choices[0]:
                        v_rand = var_vel
                    else: 
                        v_rand = var_vel

                    if choices[1]:
                        alpha_rand = var_alpha
                    else:
                        alpha_rand = var_alpha
                    # Adding reflection! To avoid running outside of max/min range.
                    if (v_ref+v_rand) > self.v_max or (v_ref+v_rand) <= 0:
                        v_ref = v_ref - v_rand
                    else:
                        v_ref = v_ref + v_rand
                    
                    if (alpha_ref + alpha_rand) > self.alpha_max or (alpha_ref + alpha_rand) <= -self.alpha_max:
                        alpha_ref = alpha_ref - alpha_rand
                    else:
                        alpha_ref = alpha_ref + alpha_rand
                    
            # Book-keeping
            alphas[i] = self.alpha
            cs[i] = self.c
            alpha_refs[i] = alpha_ref
            v_refs[i]= v_ref
            states[:, :, i] = self.X
            psis[i] = self.psi
            self.coordinated_turn_motion(alpha_ref=alpha_ref, v_ref=v_ref)

        return states, alphas, cs, v_refs, alpha_refs, psis